In [1]:
# import web driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from Login_info import user as user1, password as pass1
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome('chromedriver',options=options)

In [3]:
time.sleep(3)
driver.get("https://www.linkedin.com")

time.sleep(1)

In [4]:
def Login(wd, user,password):
    # locate email form by_class_name
    wd.find_element_by_id('session_key').send_keys(user)
    time.sleep(1)
    
    # locate password form by_class_name
    wd.find_element_by_id('session_password').send_keys(password)
    
    # locate submit button by_class_name
    wd.find_element_by_class_name('sign-in-form__submit-button').click()
    time.sleep(1)

In [5]:
def Search_company(wd, company):
    # Search bar
    search = driver.find_element_by_class_name('search-global-typeahead__input.always-show-placeholder') 
    
    # Write company name
    search.send_keys(company_name)  
    
    # Wait
    time.sleep(2)    

    # Select
    search.send_keys(Keys.ENTER) 
    time.sleep(2)

In [6]:
def Posts(wd):
    time.sleep(2)
    # Click in Posts
    wd.find_elements_by_class_name('artdeco-pill.artdeco-pill--slate.artdeco-pill--2.artdeco-pill--choice.ember-view.search-reusables__filter-pill-button')[1].click() 
    time.sleep(2)
    
    # Get number of total posts
    n_posts = int(driver.find_element_by_class_name('pb2.t-black--light.t-14').text.split(' ')[0])
    last_page = 0 if n_posts%10 == 0 else 1
    n_pages = int(n_posts/10) + last_page
    print(n_pages)
 
    # String to look
    href = 'https://www.linkedin.com/feed/update/urn:li:activity:'
    posts = {}
    
    # For each page
    for j in range(n_pages):
        print(f'Page: {j+1}')
        time.sleep(2)
        # Scroll to the end
        wd.execute_script(f"window.scrollTo(0, {10000});")
        
        # Get posts url
        for i in wd.find_elements_by_xpath(f'//a[contains(@href,"{href}")]'):
            posts[i.get_attribute('href')] = 1
        
        # Click next page
        if j + 1 < n_pages:
            wd.find_elements_by_class_name('artdeco-pagination__button.artdeco-pagination__button--next.artdeco-button.artdeco-button--muted.artdeco-button--icon-right.artdeco-button--1.artdeco-button--tertiary.ember-view')[0].click()
        
    return posts
        

In [7]:
Login(driver,user1,pass1)

In [8]:
company_name = 'Applaudo Studios'  
Search_company(driver,company_name)

In [9]:
posts = Posts(driver)

38
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38


In [70]:
for post, _ in posts.items():
    driver.get(post)
    time.sleep(1)

KeyboardInterrupt: 

In [83]:
reactions = driver.find_element_by_class_name('v-align-middle.social-details-social-counts__reactions-count')
n_reactions = int(reactions.text)
reactions.click()

In [84]:
href = 'https://www.linkedin.com/in/'
reactors = {}                              
panel = driver.find_element_by_class_name('artdeco-modal__content.social-details-reactors-modal__content.ember-view')
for i in range(int(n_reactions/10)+1):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', panel)
    time.sleep(2)


In [85]:
time.sleep(2)
for i in driver.find_elements_by_xpath(f'//a[contains(@href,"{href}")]'):
    reactors[i.get_attribute('href')] = 1

In [86]:
len(reactors)

107

In [87]:
for k in reactors.keys():
    print(k)

https://www.linkedin.com/in/ilsie-mercadillo-7650b5189?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACxUsP0BixQvu-O6JdI0Vov4MjgxEmIB7PU
https://www.linkedin.com/in/boris-estrada?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAB88Zn8Bx57dWMq1N5rKipvVQPhiLw4-1Os
https://www.linkedin.com/in/rbhg?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC4AVf8B5ztHq3SfVpioX3GzNvRtdPCi4Lw
https://www.linkedin.com/in/rodrigo-hernandezdev?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC7ZXMEBNENMXbBAyQXdAvd9uCU4P7isIYU
https://www.linkedin.com/in/baltazar-m-9070b347?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAnNhFcBe74G2E1xJghtfBb7Y5I3S33g46o
https://www.linkedin.com/in/miguel-antonio-giacoman-a1112b113?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABxRLe0B5ncWWzctu4naHskPr364GplNKYU
https://www.linkedin.com/in/cesarbendeck?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADABYWEB-tMWTUoPxEwbG5xjCrihTAq4Dqk
https://www.linkedin.com/in/marlon-ch%C3%A1vez-aa6026125?miniProfileUrn=urn%3Ali%3Afs_min

In [40]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [32]:
driver.find_elements_by_class_name('social-details-reactors-tab-body')

[<selenium.webdriver.remote.webelement.WebElement (session="821b7bf4bcf9a475625c6768c391dadf", element="283d68eb-8fd4-413d-b16b-8e34585c2039")>]

In [ ]:
#driver.close()

In [38]:
driver.window_handles

['CDwindow-F5E34C804BE68B2156FAF2D977C04EF8']

In [39]:
driver.current_window_handle

'CDwindow-F5E34C804BE68B2156FAF2D977C04EF8'